# Call by Value vs. Call by Reference


With the introduction of explicit and implicit references, we will need to dive a little deeper into how function calls are handled and understand the key concepts of call by value vs. other calling conventions. Many languages provide both conventions in some form and this is something we need to understand in the context of Lettuce with mutables. 

As a result of this lecture, you will be able to understand some confusing aspects of languages like Scala (and python) behave.

### Scala Example 1

Let us start with a simple example of a scala program.


In [1]:
def foo(x: Int): Int = {
    if (x % 2 == 1)
        3 * x + 1
    else 
        x/2
}

val z = 10
foo(z) // Function call first evals the argument
//it passes the "value" of the arg into foo.

defined function foo
x: Int = 10
res0_2: Int = 5


The program above should return the value `5`. It is very easy to reason why. When we call `foo(x)`, the value of `x` is bound to `10`. The function `foo` is then called with the arugment `10` and the rest is easy to reason about.


### Scala Program A 

Consider a different program where we pass a mutable var as an argument to a function.


In [0]:
// Program A
def bar(x: Int) : Int = { // Parameters to function calls are vals
    x = 25 // Assigns x to 25
    2 * x // returns 2 * x
}

var z: Int = 12 // mutable z
bar(z) // mutable z's value is being passed to bar.

cmd0.sc:2: reassignment to val
    x = 25 // Assigns x to 25
      ^Compilation Failed

: 

The program above declares a __mutable__ variable `z`, assigns it to 12. When we try to pass it to the function `bar` as `x` and assign it to `25` inside `x`, it does not work. The program complains that that `x` is a `val` and cannot be reassigned. Let us try the same thing but in a different context.

### Scala Program B

In [1]:
// Program B
case class Wrapper(var x : Int) // field x which is mutable

def bar(z: Wrapper): Int = {
    z.x = 25 // assigns z.x to 25
    2 * z.x // return 2 * z.x
}

val w = Wrapper(0) // Let's create a wrapper with x initialized to 0
println("Before call to bar -- > w.x = " + w.x)
bar(w) // w is being "passed by value"
println("After call to bar -- > w.x = " + w.x)



Before call to bar -- > w.x = 0
After call to bar -- > w.x = 25


defined class Wrapper
defined function bar
w: Wrapper = Wrapper(25)
res0_4: Int = 50

Something very funny is happening here: 
  - In program A, we passed a mutable integer into a function `bar` but it refuses to reassign it and complains that we cannot reassign it.
  - In program B, we wrap the mutable around a `class Wrapper` and declare it/pass an instance of the wrapper `w` as a `val` but it not only allows us to reassign but also updates the value of `w.x`. 

We will understand this once we study things in the simple case of Lettuce.

## Call By Value

Call by value means that whenever we call a function  on an argument,

~~~
   funcCalled (argument)
~~~

the argument is (fully) evaluated and its value is passed into the function call.


Consider the following Lettuce program:

~~~

let x = 10 in 
  let f = function (z) 
          2 * z in 
      f (x + 20)

~~~

There is a function call to `f` at the very last line with argument `x + 20`. The call by value semantics of Lettuce tells us to do the following:

- First evaluate the argument: it evaluates to the number 30
- Next, call the function on the value of the argument. 
   - The formal parameter `z` is now `30`
   - The function returns a `60`.
   
   
Call by value is straightforward to reason about once you understand this principle.  What will the following Lettuce program evaluate to?

~~~
   let bar = function (x) 
           let dummy1 = assignref(x, 25) in 
              2 * deref(x)
       in 
       let z = newref(12) in 
          bar(z)
~~~


Once again there is a function call `bar(z)`. The rule for call by value is to evaluate the argument.
- The value of the argument `z` is `Reference(0)`: a reference to a memory cell 0, which has the value 12 in it.
- The code now executes and assigns to the reference that was passed in (now bound to the formal parameter z)
- As a result, the contents of the cell `0` in the memory are updated to `25`.
- The result of the program is 50.


In other words, call by value is able to achieve side effects when the value that is being passed in to a function is a pointer.


### Scala always does call by value as default.

Scala supports two basic conventions: Call by value and  somewhat esoteric calling conventions such as  __call by name__ and __call by need__ (using call by name and lazy values) for which you have to use special syntax. Let us ignore call by name and focus just on the call by value.

Let us go back to program A above (recalled here):

~~~
def bar(x: Int) : Int = {
    x = 25
    2 * x
}

var z = 12
bar(z)
~~~

It fails for two reasons: 
- The parameters for functions in scala are always vals and thus immutable. Thus the parameter `x` for function `bar` in the code above is an immutable and therefore it is not reassignable.
- The value of `z` in the call to `bar(z)` is `12`. It is passed as a number `12` after it is evaluated.


There are languages where this program would be accepted but still not work. eg., python. Try the same
version of the program below in Python3:

~~~
def bar(x): 
   x = 25
   return 2 * x

z = 12
t = bar(z)
print('t = ', t)
print('z= ' , z)
~~~

Python will print

~~~
t = 50
z = 12
~~~

In other words, in python3, 
- We treat function parameters as var. Therefore, the assignment `x = 25` in the function `bar(x)` is not a problem.
- However, we still do call by value. Therefore, the call `t = bar(z)` passes the value `12` and not a reference to the parameter `z`.



In a similar vein, the Lettuce program with implicit refs will also fail:

~~~
let bar = function (x) 
    let d1 = assignVar(x, 25) in 
        2 * x
   in 
 let var z = 12 in 
    bar(z)
~~~

In the Lettuce language with implicit references, we still do call by value. 
- We evaluate the argument to 
`bar(z)`, which looks in the abstract syntax as `FunCall( Ident("bar"), Ident("z"))`.
  - `Ident("z")` in the semantics of Ident, evaluates not to the reference but to the actual contents of the memory, which is `12`.
  - The call to `bar` fails because, we try to do `assignVar` on an identifier  `x` that is not a reference.
  
  
How do we then explain the behavior of Program B (recalled below):

~~~
case class Wrapper(var x : Int)

def bar(z: Wrapper): Int = {
    // z = Wrapper(1)
    z.x = 25
    2 * z.x
}

val w = Wrapper(0)
println("Before call to bar -- > w.x = " + w.x)
bar(w)
println("After call to bar -- > w.x = " + w.x)
~~~

Once again, scala will do call by value. Therefore the call to `bar(w)` passes the object `w` we just created by value.

- However, the value of an object (something declared as a class or object) in scala is a pointer/reference to the object. Therefore, `w` is passed in as some kind of a `Reference( ..wherever w is stored..)`. As a result, we are allowed to access `z.x` in the program which is a `var` and is in fact the very same thing as `w.x` from outside thef unction call. 

## Why special treatment for objects?

Languages like Scala (and Java, Python etc..) treat  objects differently from basic types such as integers, floats and chars.  The basic difference is that the `value` of an integer, float or a char in the context of "call by value" is itself, whereas the "value" ascribed to an object in "call by value" is not a copy of the object but a reference to the contents of the object. You can see the difference in this program directly.

In [4]:
class Wrapper(var x: Int)

def bar(w: Wrapper): Int = {
    println("Inside bar: Passed in value w = " + w)
    w.x
}

def foo(x: Int): Int = {
    println("Inside foo: passed in x = " + x)
    x
}

val x1 = bar(new Wrapper(10))
val x2 = foo(20)



Inside bar: Passed in value w = ammonite.$sess.cmd3$Helper$Wrapper@62eb1af6
Inside foo: passed in x = 20


defined class Wrapper
defined function bar
defined function foo
x1: Int = 10
x2: Int = 20

Languages like C and C++ do not do this kind of special treatment when you ask them to call by value they will make
a copy of everything to implement call by value. This can be quite expensive.


~~~

struct BigDataStruct {
   
   int w;
   char z;
   int x;
   std::vector<int> vec;
   
   BigDataStruct(int w_, char z_, int x_): w_(w), z_(z), x_(x) {};
   
};


int foo( BigDataStruct bds0){
     bds0.w = 20;
     bds0.z = 'c'
     bds0.x = 45;
     bds0.vec = std::vector<int>({10, 20, 45,50, 69})
     return 0;
}


int main(){
    BigDataStruct bds(10, 20, 45);
    foo (bds) // CALL by value: Make a full copy of every field of bds into a new struct and call foo.
    // Even though foo modifies bds0, none of the changes reflect back on bds.
}

~~~

To mitigate this, C/C++ allows us to do call by reference as well as call by value.


## Call by Reference

In call by reference semantics, the value of the parameters are not passed but rather the parameters are passed as references.  This was originally implemented in fortran. Let us take an example in "fortran"-like syntax.

~~~
SUBROUTINE SWAP( X , Y) 
   TMP = X
   X  = Y
   Y = TMP
   
A = 45
B = 55
SWAP(A,B)
~~~

Here, we define a "subroutine" called SWAP with two arguments, X and Y: it exchanges their values. In regular call by value semantics, this program has no effect on A and B outside the function call. This is because, in call by value, SWAP is called on the numbers 45 and 55. In call by reference,  the call to the function SWAP maps the parameter X to A and Y to B. 

Therefore, the assignments that happen to X, Y inside the function reflect on the variables A, B in the caller of the function.

C++ supports call by reference  by placing an `&` in front of the parameter in the function definition as below:

~~~
void swap (int & x, int & y){
   int tmp = x;
   x = y;
   y = tmp;
}

int main(){
  int a = 45;
  int b = 55;
  swap(a,b);
  // a is  now assigned to 55 and b is now assigned to 45.
}
~~~

Call by reference is useful because it allows the following advantages:
- Allow "return parameters" which are assigned inside the function as multiple return values.
- Allow lower cost passing of large data structures since call by value will perform a copy.

Going back to the previous example, we can write the function foo with call by reference. This stops
the compiler from performing a copy of the entire BigDataStruct contents when foo is called. 

~~~
int foo( BigDataStruct & bds0){
     bds0.w = 20;
     bds0.z = 'c'
     bds0.x = 45;
     bds0.vec = std::vector<int>({10, 20, 45,50, 69})
     return 0;
}
~~~

## Implementing Call by Reference Semantics in Lettuce


We would like to implement something a call by reference semantics in Lettuce. For instance, we would like to support programs like this 

~~~ 
let bar = function ( & x) 
   let dummy = assignVar(x, x + x) in 
      x + 20 
 in 
   let var y  = 15 in 
     let z = bar(y) in 
        y
~~~


We place an ampersend (`&`) before the `x` in the function definition for `bar` to state that we would like `x` to be passed by reference in the function bar. As a result, we would like the call `bar(y)` to pass the parameter `y` by reference and therefore, the assignment to `x` inside function `bar` should assign `y` and the program itself should return the value `30`.

How do we achieve something like this?
- First, we have to create special flag that tells us whether functions expect arguments to be passed by reference or by value. To do so, we will modify the function definitions like thus:

$$\mathbf{Expr}\ \Rightarrow\ \text{FunDef}(\mathbf{String}, \mathbf{Expr}, \mathbf{Boolean}) $$

The extra $\mathbf{Boolean}$ field is set to true for call by value and false for call by reference. The rest of the grammar is the same as that of Lettuce with implicit references.



$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & \color{red}{FunDef( \mathbf{Identifier}, \mathbf{Expr},\mathbf{Boolean})}& \text{function (identifier-formal-parameter) expr with boolean flag}\\
 & & &  \text{that is true for call by value and false for call by ref.} \\[5pt]
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & {LetVar}(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let var stmt -- compare to let binding.}\\
 & | & {AssignVar}(\mathbf{Identifier}, \mathbf{Expr}) & \text{assign a var to a value. }
\end{array}$$

Note that the only change  is in the function definition, where we note the parameter passing convention.


How do we implement `eval` for such a language? 

First, we extend the closures to also track whether they are call by value closures or call by reference closures.

$$\begin{array}{rcl}
\mathbf{Value}\ &\Rightarrow&\ \text{NumValue} (\mathbf{Double}) \\
& \Rightarrow & \text{BoolValue}(\mathbf{Boolean}) \\
& \Rightarrow & \text{CallByValueClosure}(\mathbf{String}, \mathbf{Expr}) \\
& \Rightarrow & \text{CallByReferenceClosure}(\mathbf{String}, \mathbf{Expr}) \\
& \Rightarrow & \text{Reference}(\mathbf{Address})\\
& \Rightarrow & \text{error} \\
\end{array}$$


Next,  we define two types of eval function: `eval` (as before) and `evalRef` (specially for handling call by reference). 


Recall the rule for identifiers from Lettuce with implicit references:
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array}\ \ \text{(#3)}} $$
$$\newcommand\eval{\mathbf{eval}}$$

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j), \texttt{lookupCell}(s, j) = v}{ \eval(\texttt{Ident(x)}, \sigma, s) = (v, s) }{ident-var-ok}$$

The rule says "suppose we wish to evaluate an expression of the form `Ident(x)`"
- __If__ `x` is currently a reference to memory address `j` and
  - looking up the value of address `j` in store `s` yields the value `v`,
- __Then__ return the value `v` and store `s`.


Note that `evalRef` is the same as eval for all expressions except for Ident:

$$ \semRule{ e \ \text{is not of the form}\ \texttt{Ident}(x),\ \eval(e, \sigma, s) = (v,s') }{\textbf{evalRef}(e, \sigma, s) = (v, s') }{evalref-same-as-eval}$$.

However, for Ident expressions, evalRef differs from eval as follows:

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = v}{ \textbf{evalRef}(\texttt{Ident(x)}, \sigma, s) = (v, s) }{eval-ref-ident-ok}$$

In other words, evalRef does not do the "double indirection" for pointers. It returns whatever `x` maps to in $\sigma$ as is.

Next, function definitions will be as follows (for both eval and evalRef):


$$\semRule{}{\eval(\texttt{FunDef}(x, e, true), \sigma, s) = \text{CallByValueClosure}(x, e) }{(fundef-call-by-val)}$$


$$\semRule{}{\eval(\texttt{FunDef}(x, e, false), \sigma, s) = \text{CallByReferenceClosure}(x, e) }{(fundef-call-by-ref)}$$

Finally, all the action now happens when we call a function. Here are the call by value semantics.

$$\semRule{\eval(e_1, \sigma, s) = (v_1, s_1),\ v_1 \in \text{CallByValueClosure}(x, \texttt{fbody}, \sigma_{cl}),\ \eval(e_2, \sigma, s_1) = (v_2, s_2),\ v_2 \not=\ \mathbf{error}}{\eval(\texttt{FunCall}(e_1, e_2), \sigma, s) = \eval(\texttt{fbody}, \sigma_{cl} \circ [ x \mapsto v_2], s_2) }{(funcall-by-value)}$$

Next, here is the call by reference semantics:


$$\semRule{\eval(e_1, \sigma, s) = (v_1, s_1),\ v_1 \in \text{CallByReferenceClosure}(x, \texttt{fbody}, \sigma_{cl}),\ \mathbf{\color{red}{evalRef}}(e_2, \sigma, s_1) = (v_2, s_2),\ v_2 \not=\ \mathbf{error}}{\eval(\texttt{FunCall}(e_1, e_2), \sigma, s) = \eval(\texttt{fbody}, \sigma_{cl} \circ [ x \mapsto v_2], s_2) }{(funcall-by-reference)}$$

Notice the main difference boils down to a minor detail. We will simply use $\mathbf{evalRef}$ to evaluate for the argument of the call rather than just $\mathbf{eval}$. 

In [1]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr, callByValue: Boolean) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// Let Var
case class LetVar(x: String, e1: Expr, e2: Expr) extends Expr

// Assign Var
case class AssignVar(x: String, e: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetVar
defined class AssignVar

In [6]:
// Copy from the case for implicit references
sealed trait Value
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class CallByValueClosure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
case class CallByReferenceClosure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
case class Reference(j: Int) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

/*3. Immutable Store */

case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
    
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
    
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }
    

defined trait Value
defined class NumValue
defined class BoolValue
defined class CallByValueClosure
defined class CallByReferenceClosure
defined class Reference
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell

In [8]:
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x) {
                val v = env(x)
                v match {
                    case Reference(j) => {
                        val v1 = lookupCellValue(store, j)
                        (v1, store)           
                    }
                    case _ => (v, store)
                }  
            } else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e, true) => {
            (CallByValueClosure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunDef(x, e, false) => {
            (CallByReferenceClosure(x, e, env), store) // Return a closure with the current enviroment.
        }
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            v1 match {
                case CallByValueClosure(x, closure_ex, closed_env) => {
                    val (v2, store2) = evalExpr(e2, env, store1)
                    // First extend closed_env by binding x to v2                    
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case CallByReferenceClosure(x, closure_ex, closed_env) => {
                    // THIS IS THE BIG CHANGE WE MAKE
                    val (v2, store2) = evalExprRef(e2, env, store1)
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        
        
        case AssignVar(x, e) => {
            val (v1, store1) = evalExpr(e, env, store)
            val v2 =  if (env contains x) 
                        env(x)
                      else 
                        throw new IllegalArgumentException(s"Undefined identifier $x")
            v2 match {
                case Reference(j) => {
                    val store3 = assignToCell(store1, j, v1)
                    (v1, store3)
                }
                case _ => throw new IllegalArgumentException(s"AssignVar applied to argument that is not a mutable var")
                
            }
        }
        
        case LetVar(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (store2, j) = createNewCell(store1, v1)
            val newEnv = env + (x -> Reference(j))
            evalExpr(e2, newEnv, store2)
        }
        
    }

}

// evalExprRef simply defaults to evalExpr unless the expr is of the form ident(x)

def evalExprRef(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) =  e match {
    
    case Ident(x) => {
        if (env contains x) {
            (env(x), store)
        } else {
            throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    }
    
    case _ => evalExpr(e, env, store)
}



def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}
    

defined function evalExpr
defined function evalExprRef
defined function evalProgram

In [12]:
val x = Ident("x")
val y = Ident("y")
val bar = Ident("bar")
val dum = Ident("dummy")

/* 
  let bar = function (& x) 
           let d1 = assignVar(x, x + x) in 
              x + 20
  in
     let var y = 20 in 
     let d2 = bar(y) in 
     let d3 = bar(y) in 
         y
         
  # Expected result for call by reference is 80
            
*/

val fbody = Let("d1", AssignVar("x", Plus(x,x)), Plus(x, Const(20)))
val fdef = FunDef("x", fbody, false)
val lvar3 = Let("d3", FunCall(bar, y), y)
val lvar2 = Let("d2", FunCall(bar, y), lvar3)
val lvar1 = LetVar("y", Const(20), lvar2)
val lbar = Let("bar", fdef, lvar1)
val prog = TopLevel(lbar)

val res = evalProgram(prog)
println("result = " + res)

result = NumValue(80.0)


x: Ident = Ident("x")
y: Ident = Ident("y")
bar: Ident = Ident("bar")
dum: Ident = Ident("dummy")
fbody: Let = Let(
  "d1",
  AssignVar("x", Plus(Ident("x"), Ident("x"))),
  Plus(Ident("x"), Const(20.0))
)
fdef: FunDef = FunDef(
  "x",
  Let(
    "d1",
    AssignVar("x", Plus(Ident("x"), Ident("x"))),
    Plus(Ident("x"), Const(20.0))
  ),
  false
)
lvar3: Let = Let("d3", FunCall(Ident("bar"), Ident("y")), Ident("y"))
lvar2: Let = Let(
  "d2",
  FunCall(Ident("bar"), Ident("y")),
  Let("d3", FunCall(Ident("bar"), Ident("y")), Ident("y"))
)
lvar1: LetVar = LetVar(
  "y",
  Const(20.0),
  Let(
    "d2",
    FunCall(Ident("bar"), Ident("y")),
    Let("d3", FunCall(Ident("bar"), Ident("y")), Ident("y"))
  )
)
lbar: Let = Let(
  "bar",
  FunDef(
    "x",
    Let(
      "d1",
      AssignVar("x", Plus(Ident("x"), Ident("x"))),
      Plus(Ident("x"), Const(20.0))
    ),
    false
  ),
  LetVar(
    "y",
    Const(20.0),
    Let(
      "d2",
      FunCall(Ident("bar"), Ident("y")),
      Let("

In [13]:
val x = Ident("x")
val y = Ident("y")
val bar = Ident("bar")
val dum = Ident("dummy")

/* 
  let bar = function (x) 
           let d1 = assignVar(x, x + x) in 
              x + 20
  in
     let var y = 20 in 
     let d2 = bar(y) in 
     let d3 = bar(y) in 
         y
         
  # Expected result for call by value is ERROR
            
*/

val fbody = Let("d1", AssignVar("x", Plus(x,x)), Plus(x, Const(20)))
val fdef = FunDef("x", fbody, true)
val lvar3 = Let("d3", FunCall(bar, y), y)
val lvar2 = Let("d2", FunCall(bar, y), lvar3)
val lvar1 = LetVar("y", Const(20), lvar2)
val lbar = Let("bar", fdef, lvar1)
val prog = TopLevel(lbar)

val res = evalProgram(prog)
println("result = " + res)

: 